In [1]:
import pg8000          #pg8000 access SQL databases
import pandas as pd    #pandas will be needed to work in a dataframe
import os

In [2]:
#code from Agata
#these are nice functions to open LIMS, make a query and then close LIMS after

def _connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    conn = pg8000.connect(user=user, host=host, database=database, password=password, port=port)
    return conn, conn.cursor()

def _select(cursor, query):
    cursor.execute(query)
    columns = [ d[0] for d in cursor.description ]
    return [ dict(zip(columns, c)) for c in cursor.fetchall() ]

def limsquery(query, user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    """A function that takes a string containing a SQL query, connects to the LIMS database and outputs the result."""
    conn, cursor = _connect(user, host, database, password, port)
    try:
        results = _select(cursor, query)
    finally:
        
        #THESE ARE IMPORTANT!!!!!!
        #Every query needs to be closed when done
        cursor.close()             
        conn.close()
    return results


#this last function will take our query results and put them in a dataframe so that they are easy to work with
def get_lims_dataframe(query):
    '''Return a dataframe with lims query'''
    result = limsquery(query)
    try:
        data_df = pd.DataFrame(data=result, columns=result[0].keys())
    except IndexError:
        print "Could not find results for your query."
        data_df = pd.DataFrame()
    return data_df

## Find how many cells took longer to find instanteous threshold than they did to find rheobase (ie more SSFINEST than LSFINEST sweeps).

In [3]:
query = "SELECT cell.name AS cell_name, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C1LSFINEST150112%%' THEN 1 ELSE NULL END) AS longsquare, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C1SSFINEST150112%%' THEN 1 ELSE NULL END) AS shortsquare \
FROM specimens cell JOIN ephys_sweeps ess ON cell.id = ess.specimen_id \
JOIN ephys_stimuli ON ess.ephys_stimulus_id = ephys_stimuli.id \
WHERE cell.patched_cell_container NOTNULL \
GROUP BY cell.name"

df2 = get_lims_dataframe(query)
df2.head()

,shortsquare,longsquare,cell_name
0,0,4,Slc32a1-IRES-Cre;Ai14-326812.04.02.05
1,3,1,Slc32a1-IRES-Cre;Ai14-305535.06.02.01
2,6,5,Chrna2-Cre_OE25;Ai14-379154.02.02.01
3,5,10,H18.03.010.11.15.14
4,8,0,Vip-IRES-Cre;Ai14-342919.04.02.01


In [4]:
df2[df2['shortsquare'] > df2['longsquare']].count()

shortsquare    4436
longsquare     4436
cell_name      4436
dtype: int64

## Show the distribution of hemispheres all the cells patched in 2017 with the age and sex of the mice used.

In [5]:
query = "SELECT * FROM ephys_roi_results"
query2= "SELECT * FROM specimens"
query3= "SELECT * FROM donors"
query4= "SELECT * FROM rna_amplification_inputs"
query5= "SELECT * FROM rna_amplifications"
query6= "SELECT * FROM rna_amplification_sets"
query7= "SELECT * FROM specimen_tags_specimens"

df3 = limsquery(query)
df4 = limsquery(query2)
df5 = limsquery(query3)
df6 = limsquery(query4)
df7 = limsquery(query5)
df8 = limsquery(query6)
df9 = limsquery(query7)

print df3[0].keys()
print " "
print df4[0].keys()
print " "
print df5[0].keys()
print " "
print df6[0].keys()
print " "
print df7[0].keys()
print " "
print df8[0].keys()
print " "
print df9[0].keys()


['rig_name', 'ephys_qc_criteria_id', 'failed_bad_rs', 'updated_at', 'storage_directory', 'electrode_0_pa', 'input_resistance_mohm', 'id', 'stage2_reviewer_id', 'blowout_mv', 'failed_other', 'sampling_rate', 'input_access_resistance_ratio', 'failed_no_seal', 'workflow_state', 'ephys_specimen_roi_plan_id', 'initial_access_resistance_mohm', 'qc_notes', 'recording_date', 'created_at', 'seal_gohm', 'published_at', 'failed_clogged_pipette', 'stage1_reviewer_id', 'failed_electrode_0', 'notes']
 
['cell_depth', 'ephys_roi_result_id', 'parent_y_coord', 'reference_space_id', 'updated_at', 'cell_label', 'preparation_method_id', 'parent_x_coord', 'task_id', 'location_id', 'id', 'cortex_layer_id', 'plane_of_section_id', 'frozen_at', 'flipped_specimen_id', 'data', 'pinned_radius', 'rna_integrity_number', 'histology_well_name', 'created_by', 'priority', 'parent_id', 'ephys_start_time_sec', 'project_id', 'alignment3d_id', 'carousel_well_name', 'patched_cell_container', 'updated_by', 'cell_prep_id', 'b

In [6]:
query = "SELECT err.recording_date, s.donor_id, s.hemisphere_id, d.age_id, d.gender_id \
FROM ephys_roi_results err \
JOIN specimens s ON s.ephys_roi_result_id = err.id \
JOIN donors d ON s.donor_id = d.id \
WHERE s.hemisphere_id = 1.0 OR s.hemisphere_id = 2.0 \
AND (err.recording_date > '2017-01-01' AND err.recording_date < '2017-12-31')"

df = get_lims_dataframe(query)
df.tail()

,gender_id,donor_id,hemisphere_id,recording_date,age_id
3171,2,643830492,2,2017-10-27 21:09:28,298414236
3172,1,656437761,1,2018-01-08 20:13:11,114
3173,1,657607092,1,2018-01-23 23:05:47,298414237
3174,1,657607092,1,2018-01-23 23:41:09,298414237
3175,1,673427470,1,2018-03-21 17:03:38,298414239


## Look for cells that do not have a qc reviewer and failed qc for project T301. What are the cell level qc failures? (access, gohm seal, blowout, input resistance)

In [7]:
query= "SELECT err.stage1_reviewer_id, err.workflow_state, err.stage2_reviewer_id, proj.code, \
err.failed_bad_rs, err.failed_electrode_0, err.failed_no_seal, err.blowout_mv, \
err.initial_access_resistance_mohm \
FROM ephys_roi_results err JOIN specimens s ON s.ephys_roi_result_id = err.id \
JOIN projects proj ON s.project_id = proj.id \
WHERE proj.code = 'T301' \
AND workflow_state = 'auto_failed'"

df = get_lims_dataframe(query)
df


,code,failed_bad_rs,workflow_state,stage2_reviewer_id,blowout_mv,stage1_reviewer_id,failed_electrode_0,initial_access_resistance_mohm,failed_no_seal
0,T301,False,auto_failed,NaN,NaN,NaN,True,NaN,None
1,T301,False,auto_failed,NaN,NaN,NaN,False,NaN,None
2,T301,False,auto_failed,NaN,NaN,NaN,True,NaN,None
3,T301,False,auto_failed,NaN,NaN,NaN,True,NaN,None
4,T301,False,auto_failed,NaN,NaN,NaN,False,NaN,None
5,T301,False,auto_failed,NaN,NaN,NaN,True,NaN,None
6,T301,False,auto_failed,NaN,NaN,NaN,True,NaN,None
7,T301,False,auto_failed,NaN,NaN,NaN,True,NaN,None
8,T301,False,auto_failed,NaN,NaN,NaN,True,NaN,None
9,T301,False,auto_failed,NaN,NaN,NaN,True,NaN,None


In [8]:
query = "SELECT specimens.name FROM specimens"

df2 = get_lims_dataframe(query)
df2.head()

,name
0,Ndnf-IRES2-dgCre;Ai14-280612.06.02
1,Chrna2-Cre_OE25;Ai14(IVSCC)-312984
2,Sst-IRES-Cre;Ai140;Pvalb-2A-FlpO;Ai65F-303812
3,Htr3a-Cre_NO152;Ai14-288785
4,Chrna2-Cre_OE25;Ai14(IVSCC)-312987


# Making a user report

In [9]:
query = "SELECT s.parent_id, s.patched_cell_container, err.recording_date \
FROM specimens s \
JOIN projects proj ON s.project_id = proj.id \
JOIN ephys_roi_results err ON s.ephys_roi_result_id = err.id \
WHERE proj.code <> 'mMPATCH' AND \
s.patched_cell_container IS NOT NULL \
AND err.recording_date > '2017-10-01'"

patched_cell_df = get_lims_dataframe(query)

patched_cell_df.head()


,parent_id,patched_cell_container,recording_date
0,639546677,P8S4_171002_351_A01,2017-10-02 17:55:51
1,639546643,P9S4_171002_401_A01,2017-10-02 18:00:47
2,639546677,P8S4_171002_352_A01,2017-10-02 18:17:44
3,639517805,P2S4_171002_051_A01,2017-10-02 18:29:21
4,639546677,P8S4_171002_353_A01,2017-10-02 18:47:06


In [10]:
users = {'P1':'Kristen', 'P2':'Rusty', 'P8':'Lindsay', 'P9':'Lisa', 'PA':'Ram', 'PB':'DiJon'}
lindsay = patched_cell_df[patched_cell_df["patched_cell_container"].str.contains('P8')]
lindsay

,parent_id,patched_cell_container,recording_date
0,639546677,P8S4_171002_351_A01,2017-10-02 17:55:51.000
2,639546677,P8S4_171002_352_A01,2017-10-02 18:17:44.000
4,639546677,P8S4_171002_353_A01,2017-10-02 18:47:06.000
7,639621892,P8S4_171002_354_A01,2017-10-02 20:39:22.000
8,639621892,P8S4_171002_355_A01,2017-10-02 21:12:16.000
13,639621770,P8S4_171002_356_A01,2017-10-02 22:10:03.000
18,639621770,P8S4_171002_357_A01,2017-10-02 22:35:02.000
22,639789777,P8S4_171003_351_A01,2017-10-03 16:37:01.000
24,639789777,P8S4_171003_352_A01,2017-10-03 16:59:16.000
31,639796677,P8S4_171003_353_A01,2017-10-03 18:07:38.000


In [11]:
def cell_count(P_number, df):
    user = df[df["patched_cell_container"].str.contains(P_number)]
    return user["parent_id"].count()
    

In [12]:
print cell_count('P8',patched_cell_df)

1490


In [13]:
query = "SELECT s.parent_id, s.patched_cell_container, err.recording_date, ra.failed \
FROM specimens s \
JOIN projects proj ON s.project_id = proj.id \
JOIN ephys_roi_results err ON s.ephys_roi_result_id = err.id \
JOIN rna_amplification_inputs rai on rai.sample_id = s.id \
JOIN rna_amplifications ra on ra.id = rai.rna_amplification_id \
WHERE proj.code <> 'mMPATCH' AND \
s.patched_cell_container IS NOT NULL \
AND ra.failed = 'False' \
AND err.recording_date > '2017-10-01'"

df2 = get_lims_dataframe(query)
df2

#FROM specimens cell \
#LEFT JOIN rna_amplification_inputs rai ON rai.sample_id = cell.id \
#LEFT JOIN rna_amplifications ra ON ra.id = rai.rna_amplification_id \

#from cells cell
#join specimens slice on slice.id = cell.slice_specimen_id 

#left join rna_amplification_inputs rai on rai.sample_id = cell.cell_specimen_id
#left join rna_amplifications ra on ra.id = rai.rna_amplification_id 

,parent_id,patched_cell_container,recording_date,failed
0,639546677,P8S4_171002_351_A01,2017-10-02 17:55:51.000,False
1,639546643,P9S4_171002_401_A01,2017-10-02 18:00:47.000,False
2,639517805,P2S4_171002_051_A01,2017-10-02 18:29:21.000,False
3,639546677,P8S4_171002_353_A01,2017-10-02 18:47:06.000,False
4,639517805,P2S4_171002_052_A01,2017-10-02 19:03:11.000,False
5,639627116,PAS4_171002_451_A01,2017-10-02 20:35:10.000,False
6,639621892,P8S4_171002_354_A01,2017-10-02 20:39:22.000,False
7,639621892,P8S4_171002_355_A01,2017-10-02 21:12:16.000,False
8,639626809,P9S4_171002_402_A01,2017-10-02 21:31:42.000,False
9,639621802,P2S4_171002_054_A01,2017-10-02 21:55:43.000,False


In [14]:
print cell_count('P8',df2)

1239


In [24]:
def find_slices(specimen):
    """Return the names of slices used from a particular mouse prep.
     
    Parameters
    ----------
    specimen : integer corresponding to mouse specimen name
    
    Returns
    -------
    strings of slice names
    """
   
    for file in os.listdir('//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/'):
        if str(specimen) in file:
            print file[-20:-8]

In [25]:
find_slices(447290)

447290.05.02
447290.05.01
447290.04.01
447290.03.01
447290.03.02
447290.04.02


In [26]:
query = "SELECT s.patched_cell_container, err.recording_date \
FROM specimens s \
JOIN projects proj ON s.project_id = proj.id \
JOIN ephys_roi_results err ON s.ephys_roi_result_id = err.id \
WHERE proj.code <> 'mMPATCH' AND \
s.patched_cell_container IS NOT NULL \
AND err.recording_date > '2017-10-01'"

patched_cell_df = get_lims_dataframe(query)
patched_cell_df.head()

,patched_cell_container,recording_date
0,P8S4_171002_351_A01,2017-10-02 17:55:51
1,P9S4_171002_401_A01,2017-10-02 18:00:47
2,P8S4_171002_352_A01,2017-10-02 18:17:44
3,P2S4_171002_051_A01,2017-10-02 18:29:21
4,P8S4_171002_353_A01,2017-10-02 18:47:06


In [27]:
query2 = "SELECT s.patched_cell_container, err.recording_date, ra.failed \
FROM specimens s \
JOIN projects proj ON s.project_id = proj.id \
JOIN ephys_roi_results err ON s.ephys_roi_result_id = err.id \
JOIN rna_amplification_inputs rai on rai.sample_id = s.id \
JOIN rna_amplifications ra on ra.id = rai.rna_amplification_id \
WHERE proj.code <> 'mMPATCH' AND \
s.patched_cell_container IS NOT NULL \
AND ra.failed = 'False' \
AND err.recording_date > '2017-10-01'"

QC_passed_df = get_lims_dataframe(query2)
QC_passed_df.head()

,failed,patched_cell_container,recording_date
0,False,P8S4_171002_351_A01,2017-10-02 17:55:51
1,False,P9S4_171002_401_A01,2017-10-02 18:00:47
2,False,P2S4_171002_051_A01,2017-10-02 18:29:21
3,False,P8S4_171002_353_A01,2017-10-02 18:47:06
4,False,P2S4_171002_052_A01,2017-10-02 19:03:11


In [28]:
query3 = "WITH do_63x(cell_id, go) AS (SELECT DISTINCT cell.id, array_to_string(array_agg(DISTINCT tag.name), ' _AND_ ') \
FROM specimens cell JOIN ephys_roi_results err ON err.id = cell.ephys_roi_result_id \
JOIN specimen_tags_specimens sptagsp ON sptagsp.specimen_id = cell.id \
JOIN specimen_tags tag ON tag.id = sptagsp.specimen_tag_id AND tag.id in (602120185,602122082) \
GROUP BY cell.id ORDER BY 1) \
SELECT s.patched_cell_container, err.recording_date, do_63x.go \
FROM specimens s \
JOIN projects proj ON s.project_id = proj.id \
LEFT JOIN do_63x ON do_63x.cell_id = s.id \
JOIN ephys_roi_results err ON s.ephys_roi_result_id = err.id \
WHERE proj.code <> 'mMPATCH' AND \
s.patched_cell_container IS NOT NULL \
AND err.recording_date > '2017-10-01'"

image_df = get_lims_dataframe(query3)
image_df.head()

,go,patched_cell_container,recording_date
0,63x go,P8S4_171002_351_A01,2017-10-02 17:55:51
1,63x go,P9S4_171002_401_A01,2017-10-02 18:00:47
2,63x no go,P8S4_171002_352_A01,2017-10-02 18:17:44
3,63x go,P2S4_171002_051_A01,2017-10-02 18:29:21
4,63x no go,P8S4_171002_353_A01,2017-10-02 18:47:06


In [29]:
def cell_count(P_number, df):
    """Return the number of cells in a specified dataframe for a specified user.
     
    Parameters
    ----------
    P_number : A string. Corresponds to the user's P number in the form of P1, P2, P3, etc.
    df: a pandas dataframe
    
    Returns
    -------
    An integer cell number
    """
    user = df[df["patched_cell_container"].str.contains(P_number)]
    return user["patched_cell_container"].count()
    

In [30]:
print "Total cells patched:", cell_count('P8',patched_cell_df)
print "Cells that passed RNA seq QC:", cell_count('P8',QC_passed_df)
print "QC pass percentage:", float((cell_count('P8',QC_passed_df)))/float((cell_count('P8',patched_cell_df)))

Total cells patched: 1490
Cells that passed RNA seq QC: 1239
QC pass percentage: 0.831543624161


In [31]:
query = "WITH do_63x(cell_id, go) AS (SELECT DISTINCT cell.id, array_to_string(array_agg(DISTINCT tag.name), ' _AND_ ') \
FROM specimens cell JOIN ephys_roi_results err ON err.id = cell.ephys_roi_result_id \
JOIN specimen_tags_specimens sptagsp ON sptagsp.specimen_id = cell.id \
JOIN specimen_tags tag ON tag.id = sptagsp.specimen_tag_id AND tag.id in (602120185,602122082) \
GROUP BY cell.id ORDER BY 1) \
SELECT s.patched_cell_container, err.recording_date, do_63x.go AS go_no_go_63x, ra.failed AS RNA_seq_QC_Failed, s.name \
FROM specimens s \
LEFT JOIN projects proj ON s.project_id = proj.id \
LEFT JOIN do_63x ON do_63x.cell_id = s.id \
LEFT JOIN ephys_roi_results err ON s.ephys_roi_result_id = err.id \
LEFT JOIN rna_amplification_inputs rai on rai.sample_id = s.id \
LEFT JOIN rna_amplifications ra on ra.id = rai.rna_amplification_id \
WHERE proj.code <> 'mMPATCH' AND \
s.patched_cell_container IS NOT NULL \
AND err.recording_date > '2017-10-01'"

#AND (err.recording_date > '2017-10-01' OR err.recording_date IS NULL)
#AND err.recording_date > '2017-10-01'

master_df = get_lims_dataframe(query)
#master_df.sort_values("patched_cell_container", axis = 0, inplace = True)
master_df.head()

,rna_seq_qc_failed,patched_cell_container,go_no_go_63x,name,recording_date
0,False,P8S4_171002_351_A01,63x go,Oxtr-T2A-Cre;Ai14-351467.05.02.01,2017-10-02 17:55:51
1,False,P9S4_171002_401_A01,63x go,Oxtr-T2A-Cre;Ai14-351467.03.01.01,2017-10-02 18:00:47
2,True,P8S4_171002_352_A01,63x no go,Oxtr-T2A-Cre;Ai14-351467.05.02.02,2017-10-02 18:17:44
3,False,P2S4_171002_051_A01,63x go,Nos1-CreERT2;Sst-IRES-FlpO;Ai65-350934.04.01.01,2017-10-02 18:29:21
4,False,P8S4_171002_353_A01,63x no go,Oxtr-T2A-Cre;Ai14-351467.05.02.03,2017-10-02 18:47:06


In [32]:
query = """
WITH do_63x(cell_id, go) AS (SELECT DISTINCT cell.id, array_to_string(array_agg(DISTINCT tag.name), ' _AND_ ') 
FROM specimens cell JOIN ephys_roi_results err ON err.id = cell.ephys_roi_result_id 
JOIN specimen_tags_specimens sptagsp ON sptagsp.specimen_id = cell.id 
JOIN specimen_tags tag ON tag.id = sptagsp.specimen_tag_id AND tag.id in (602120185,602122082) 
GROUP BY cell.id ORDER BY 1), 
imgs20 AS (WITH tag_list AS (SELECT DISTINCT ims.id AS ims_id, imst.name AS tag FROM image_series ims 
JOIN image_series_image_series_tags ims2imst ON ims2imst.image_series_id = ims.id 
JOIN image_series_tags imst ON imst.id = ims2imst.image_series_tag_id 
ORDER BY 1,2) 
SELECT DISTINCT ims.id AS image_series_id, ims.workflow_state AS workflow_state, ims.specimen_id, 
ARRAY_TO_STRING(ARRAY_AGG(DISTINCT tag_list.tag), '_AND_') AS tags FROM image_series ims 
LEFT JOIN tag_list ON tag_list.ims_id = ims.id 
WHERE ims.type = 'FocalPlaneImageSeries' AND is_stack = false 
GROUP BY ims.id, ims.workflow_state, ims.specimen_id),
dendrite_type AS (select sts.specimen_id, array_to_string(array_agg(DISTINCT tag.name), ' ') as dendrite_type
from specimen_tags_specimens sts
join specimen_tags tag on sts.specimen_tag_id = tag.id
where tag.name in('dendrite type - spiny', 'dendrite type - NA', 'dendrite type - sparsely spiny', 'dendrite type - aspiny')
group by sts.specimen_id)
SELECT DISTINCT dendrite_type, s.patched_cell_container, err.recording_date, do_63x.go AS go_no_go_63x, ra.failed AS RNA_seq_QC_Failed, s.name,
'=HYPERLINK("http://lims2/focal_plane_image_series?id=' || imgs20.image_series_id || '")' as link_20x
FROM specimens s 
LEFT JOIN projects proj ON s.project_id = proj.id 
LEFT JOIN do_63x ON do_63x.cell_id = s.id 
JOIN specimens slice ON s.parent_id = slice.id
LEFT JOIN specimen_tags_specimens sts ON sts.specimen_id = s.id
LEFT JOIN ephys_roi_results err ON s.ephys_roi_result_id = err.id 
LEFT JOIN rna_amplification_inputs rai on rai.sample_id = s.id 
LEFT JOIN imgs20 ON slice.id = imgs20.specimen_id 
LEFT JOIN rna_amplifications ra on ra.id = rai.rna_amplification_id 
WHERE proj.code <> 'mMPATCH' AND 
s.patched_cell_container IS NOT NULL 
AND err.recording_date > '2017-10-01'
"""


#AND (err.recording_date > '2017-10-01' OR err.recording_date IS NULL)
#AND err.recording_date > '2017-10-01'

master_df = get_lims_dataframe(query)
#master_df.sort_values("patched_cell_container", axis = 0, inplace = True)
master_df

ProgrammingError: (u'ERROR', u'42703', u'column "dendrite_type" does not exist', u'1389', u'parse_relation.c', u'2892', u'errorMissingColumn')